In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-23"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
18309,2024-11-23,Suécia Liga De Basquete,10:00,Uppsala F,Ostersund F,1.10,6.66,139.5,1.83,1.83,-15.5,2.00,1.72,O4Oejd6b,0.909091,0.150150,0.546448,0.546448,0.059241,314.892,168.463520,0.534988,1.2,1.643168,1.369306,384.80,6.516,4.270355,0.655364,-107.0,357.756,471.934975,1.319153,0.6,1.341641,2.236068,161.20,6.604,9.732976,1.473800,-70.0,37,65,10.40,2.48,240.522,356.328,1.013277,0.000000,0.106446,-2.45,-0.490,-0.204082,0.466504,0.4,-0.066504,-0.80,-0.160,-35.375000,0.446916,0.3,-0.146916
18310,2024-11-23,Turquia Tb2L,09:00,Karamursel,Ege Universitesi,3.84,1.20,0.0,0.00,0.00,9.5,1.88,1.79,pI7ODmjC,0.260417,0.833333,inf,inf,0.093750,147.948,45.384426,0.306759,1.8,1.643168,0.912871,123.42,2.156,0.639398,0.296567,-9.0,192.160,81.859886,0.425999,1.8,1.643168,0.912871,195.92,2.816,1.326510,0.471062,15.0,51,79,2.42,2.48,0.000,187.872,0.740779,NaN,0.034681,0.00,0.000,inf,0.000000,0.0,0.000000,0.63,0.126,1.587302,0.000000,0.0,0.000000
18311,2024-11-23,Suécia Liga De Basquete,11:00,Hogsbo F,Sodertalje F,1.83,1.83,150.5,1.83,1.83,-1.5,1.91,1.77,d0BTnNa6,0.546448,0.546448,0.546448,0.546448,0.092896,314.162,126.399663,0.402339,0.6,1.341641,2.236068,256.20,4.506,1.919682,0.426028,-62.0,136.946,75.272342,0.549650,2.4,1.341641,0.559017,270.97,1.714,0.974695,0.568667,56.0,61,79,4.20,3.43,197.528,474.280,0.000000,0.000000,0.053802,-3.98,-0.796,-1.042714,0.534817,0.4,-0.134817,-1.45,-0.290,-2.862069,0.422749,0.7,0.277251
18312,2024-11-23,Suécia Liga De Basquete,13:00,MarkSjuharad F,Lund F,2.50,1.51,147.5,1.83,1.83,3.5,1.96,1.75,lz8LlqVg,0.400000,0.662252,0.546448,0.546448,0.062252,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,258.258,201.077288,0.778591,0.6,1.341641,2.236068,254.14,3.758,3.465475,0.922160,-57.0,0,97,0.00,2.62,0.000,253.702,0.349145,0.000000,0.080050,0.00,0.000,inf,0.000000,0.0,0.000000,-3.50,-0.700,-0.728571,0.000000,0.0,0.000000
18313,2024-11-23,Suíça Liga Sb Feminina,12:00,Geneve F,Riva Basket F,1.13,4.77,136.5,1.83,1.83,-12.5,1.95,1.73,dS7iD7KG,0.884956,0.209644,0.546448,0.546448,0.094599,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,287.626,274.838271,0.955540,1.2,1.643168,1.369306,142.19,3.980,3.675629,0.923525,-28.0,82,59,2.02,2.41,0.000,299.858,0.872498,0.000000,0.084545,0.00,0.000,inf,0.000000,0.0,0.000000,-1.09,-0.218,-17.293578,0.377320,0.3,-0.077320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18484,2024-11-23,Finlândia Korisliiga Feminina,12:00,Pyrinto F,Torpan Pojat Helsinki F,7.85,1.04,159.5,1.83,1.83,14.5,1.98,1.70,bVYPhtXi,0.127389,0.961538,0.546448,0.546448,0.088927,146.694,56.481118,0.385027,2.4,1.341641,0.559017,104.00,1.750,0.498046,0.284598,11.0,99.996,10.517881,0.105183,2.4,1.341641,0.559017,107.97,1.212,0.317285,0.261787,59.0,100,61,1.04,1.77,184.128,97.772,1.083329,0.000000,0.107603,2.14,0.428,16.004673,0.535684,0.6,0.064316,-1.82,-0.364,-0.109890,0.799565,0.7,-0.099565
18485,2024-11-23,Finlândia Divisão I Feminina,09:00,Lahti Basketball F,Torpan Pojat Helsinki II F,2.95,1.33,139.5,1.88,1.78,5.5,2.01,1.68,S6ttheCd,0.338983,0.75188

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
18316,11:15,Turquia Tbl,Balikesir,Fenerbahce 2,1.59,Back Home
18318,12:00,Turquia Tb2L,Akhisar Belediye,Corlu,2.12,Back Home
18328,13:00,Suécia Superettan,Blackeberg,AIK Basket,2.10,Back Home
18335,14:30,Portugal Proliga,Santo André,CAB Madeira,1.98,Back Home
18344,13:00,Sérvia Primeira Liga,Radnicki,Sloboda,1.70,Back Home
18358,18:00,Eua Ncaa,Wisc. Milwaukee,Wofford,1.69,Back Home
18360,19:00,Eua Ncaa,UMass,Temple,1.83,Back Home
18372,12:00,Grécia Liga De Elite,Hanth,Eleftheroupoli,2.41,Back Home
18381,13:00,Eua Ncaa,Cal St. Bakersfield,Florida Intl,2.05,Back Home
18391,15:00,Eua Ncaa,East Carolina,Illinois (Chi.),1.87,Back Home
